In [3]:
import pandas as pd

In [4]:
df = pd.read_csv("spotify_millsongdata.csv")

In [5]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [6]:
df.tail(5)

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [7]:
df.shape

(57650, 4)

In [8]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [9]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [10]:
df.head(10)

,artist,song,text
0,XTC,That Is The Way,Go and speak to your niece (do this do that) ...
1,Kenny Rogers,Calling Me,There's a back road leadin' off the hard-top ...
2,Violent Femmes,To The Kill,\r\n \r\nAin't had no fun \r\n \r\nall th...
3,System Of A Down,Soil,"The phoenix he helped create, \r\nOut of cont..."
4,Tom T. Hall,Soldier Of Fortune,I'll follow my dreams like a soldier of fortun...
5,Townes Van Zandt,Greensboro Woman,Greensboro woman don't you smile on me \r\nI ...
6,Toto,You Are The Flower,"You never lose a minute, if in it there is lov..."
7,NOFX,Cell Out,She asked me if I was a singer then called me ...
8,One Direction,I Want To Write You A Song,[Verse 1: Harry] \r\nI wanna write you a song...
9,Bryan White,Sittin On Go,I'm waiting on the green light to make you min...


In [11]:
df['text'][0]

"Go and speak to your niece (do this do that)  \r\nKiss your aunt on the cheek  \r\nDon't talk back, now run along  \r\nIt's gone eight, say goodnight  \r\nThat's the way that it's done  \r\nBoy and girl, girl and boy  \r\nThis is how you do it and  \r\nWho am I to reason why  \r\nWear a shirt, wear a tie  \r\nYou look smart in a suit  \r\nUse your fork in the left  \r\nStraighten up shoulders back  \r\nThis is the way that it's done\r\n\r\n"

In [12]:
df = df.sample(5000)

In [13]:
df.shape

(5000, 3)

Text Cleaning/ Text Preprocessing

In [14]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [15]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [17]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [18]:
similarity

array([[1.00000000e+00, 2.62469998e-02, 2.51366627e-03, ...,
        1.42901152e-02, 5.30166158e-03, 2.27293882e-02],
       [2.62469998e-02, 1.00000000e+00, 8.85810991e-03, ...,
        1.54414831e-02, 3.14161022e-02, 2.81143489e-02],
       [2.51366627e-03, 8.85810991e-03, 1.00000000e+00, ...,
        3.61863351e-03, 1.26832199e-02, 9.24582154e-03],
       ...,
       [1.42901152e-02, 1.54414831e-02, 3.61863351e-03, ...,
        1.00000000e+00, 3.02286565e-04, 8.05774622e-03],
       [5.30166158e-03, 3.14161022e-02, 1.26832199e-02, ...,
        3.02286565e-04, 1.00000000e+00, 4.04324836e-03],
       [2.27293882e-02, 2.81143489e-02, 9.24582154e-03, ...,
        8.05774622e-03, 4.04324836e-03, 1.00000000e+00]])

In [19]:
df[df['song'] == 'Crying Over You']

,artist,song,text


In [20]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])

    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)

    return songs

In [21]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))


In [22]:
def normalise(matrix):
    min_val = np.min(matrix)
    max_val = np.max(matrix)
    normalized_matrix = (matrix - min_val) / (max_val - min_val)
    return normalized_matrix

In [29]:
import numpy as np
print("Similarity Matrix For TF-IDF")
normalise(similarity[0])
print(similarity[0])

Similarity Matrix For TF-IDF
[1.         0.026247   0.00251367 ... 0.01429012 0.00530166 0.02272939]


In [30]:
print("Similarity Matrix For Word2Vec")
# normalise(similarity)
mat=normalise(similarity[1])
print(mat)

Similarity Matrix For Word2Vec
[0.026247   1.         0.00885811 ... 0.01544148 0.0314161  0.02811435]


In [31]:
print("Similarity Matrix For Doc2Vec")
mat=normalise(similarity[2])
print(mat)

Similarity Matrix For Doc2Vec
[0.00251367 0.00885811 1.         ... 0.00361863 0.01268322 0.00924582]


In [27]:
print("OverAll Similarity Matrix")
mat=normalise(similarity[3])
print(mat)

OverAll Similarity Matrix
[0.01288676 0.01874668 0.01027509 ... 0.00182968 0.00999372 0.00843681]


In [28]:
def normalise(matrix):
    min_val = np.min(matrix)
    max_val = np.max(matrix)
    normalized_matrix = (matrix - min_val) / (max_val - min_val)
    return normalized_matrix